# Лабораторна робота 2 з "Асиметричних криптосистем та протоколів"
## Тема: Вивчення криптосистеми RSA та алгоритму електронного підпису; ознайомлення з методами генерації параметрів для асиметричних криптосистем

**Виконали**\
Дигас Богдан, ФІ-03\
Починок Юрій, ФІ-03

In [777]:
import random
import math
import numpy as np
rand = random.SystemRandom()

In [778]:

def decomposing_number(n, a):
    exp = n - 1
    while not exp & 1:  # while exp is even
        exp >>= 1  # divide by 2
    if pow(a, exp, n) == 1:
        return True  # number is composite
    while exp < n - 1:
        if pow(a, exp, n) == n - 1:
            return True  # number is composite
        exp <<= 1  # multiply by 2
    return False  # number is probably prime


def miller_rabbin_test(n, k=20):
    for i in range(k):
        a = rand.randrange(1, n - 1)
        if not decomposing_number(n, a):
            return False  # number is composite
    return True  # number is probably prime

def bin_to_dec(bin_n):
    dec_n = 0
    res = 0
    for i in range(len(bin_n)):
        res = bin_n[len(bin_n) - i - 1] * 2 ** i
        dec_n += res
    return dec_n


In [779]:
def generate_bit_seq(n):
    seq = [0]*n
    for i in range(n):
        seq[i] = rand.randint(0, 1)
    return seq


def L20(n):
    seq = generate_bit_seq(20)
    result = [0]*n
    for i in range(20):
        result[i] = seq[i]
    for i in range(20,n):
        result[i] = result[i-3]^result[i-5]^result[i-9]^result[i-20]
    return result


In [780]:
def generate_prime_number(x):
    res = [1,0,0]
    while(miller_rabbin_test(bin_to_dec(res)) == False):
        res = L20(x)
    return res

In [781]:
res = generate_prime_number(256)
print(res)

[1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1]


In [782]:
def generate_better_prime_numbers(n):
    p = bin_to_dec(generate_prime_number(n))
    q = bin_to_dec(generate_prime_number(n))
    
    i = 1
    while(miller_rabbin_test(2*p*i + 1) == False):
        i = i + 1

    better_p = 2*p*i + 1
    
    j = 1
    while(miller_rabbin_test(2*q*j + 1) == False):
        j = j + 1

    better_q = 2*q*j + 1
    
    return better_p, better_q

In [783]:
def RSA_gen_constants():
    p,q = generate_better_prime_numbers(256)
    e = 2**16 + 1
    n = p*q
    phi_n = (p-1)*(q-1)
    d = pow(e,-1,phi_n)
    
    return p,q,n,e,d

In [784]:
class User:
    
    __private_key = None
    __key_pair = None
    
    __k = None

    def __init__(self):
        result = RSA_gen_constants()
        self.__key_pair = result[0], result[1]
        self.public_n = result[2]
        self.public_e = result[3]
        self.__private_key = result[4]
        
    
    def RSA_decrypt(self, C):
        res = pow(C, self.__private_key, self.public_n)
        return res
    
    def RSA_sign(self, M):
        res = pow(M, self.__private_key, self.public_n)
        return res
    
    def Send_Key(self, e_1, n_1):
        while(self.public_n > n_1):
            print("Regenerating keys")
            print(self.public_n," && ", n_1)
            result = RSA_gen_constants()
            self.__key_pair = result[0], result[1]
            self.public_n = result[2]
            self.public_e = result[3]
            self.__private_key = result[4]
        
        k = random.randint(1, self.public_n-1)
        print("initial k, ", hex(k))
        S = pow(k, self.__private_key, self.public_n)
        S_1 = pow(S, e_1, n_1)
        k_1 = pow(k, e_1, n_1)
        
        return k_1, S_1
    
    def Receive_Key(self, k_1, S_1, e, n):
        k = pow(k_1, self.__private_key, self.public_n)
        S = pow(S_1, self.__private_key, self.public_n)
        
        if k == pow(S, e, n):
            __k = k
            print("k is authentic, ", hex(k))
        else:
            print("k is not authentic, try again")

In [785]:
A = User()
B = User()

def RSA_encrypt(m,e,n):
    res = pow(m,e,n)
    return res

def RSA_verify(S, M, e, n):
    res = pow(S,e,n)
    return res == M


In [786]:
print(hex(A.public_n))

0x8b41386cc56ada854fea1921cec47551132e11c13961a166bf50bd0e0d9d81d8bb09a4951bae43dd66130e669927a18754ae6a92bee4e69f930d1202f32ead87d07b


## Server Key

In [787]:
server_key = int("C2C686510DBB7E57A11C92322603919B959CC5DC5C535D812B28D376860C66EF", 16)
server_e = 2**16+1

## Encryption

In [788]:
M = int("07B4F868685AB20081", 16)
print(hex(A.public_n))
print(hex(A.public_e))
print(hex(A.RSA_decrypt(M)))

0x8b41386cc56ada854fea1921cec47551132e11c13961a166bf50bd0e0d9d81d8bb09a4951bae43dd66130e669927a18754ae6a92bee4e69f930d1202f32ead87d07b
0x10001
0x7ebfa24c5ddea29e9b9bf95157229da6526ac76e46f4983ba0e6820f3edf0241cbd4ade8448fd6f073930a2b2e01fdda54ab8e970b6b840ab6daaec914b67c6c843b


## Decryption

In [789]:
C = int("07B4F868685AB20081", 16)
print(hex(A.RSA_decrypt(C)))

0x7ebfa24c5ddea29e9b9bf95157229da6526ac76e46f4983ba0e6820f3edf0241cbd4ade8448fd6f073930a2b2e01fdda54ab8e970b6b840ab6daaec914b67c6c843b


## Signature

In [790]:
M = int("07B4F868685AB20081", 16)
server_S = int("496CC7A5A8631A8279F2F15ABAE40CABAD9937CB8D97027783E914423EC7B191",16)
print(RSA_verify(server_S, M, server_e, server_key))

False


## Verification

In [791]:
M = int("07B4F868685AB20081", 16)
print(hex(A.RSA_sign(M)))
print(hex(A.public_n))
print(hex(A.public_e))

0x7ebfa24c5ddea29e9b9bf95157229da6526ac76e46f4983ba0e6820f3edf0241cbd4ade8448fd6f073930a2b2e01fdda54ab8e970b6b840ab6daaec914b67c6c843b
0x8b41386cc56ada854fea1921cec47551132e11c13961a166bf50bd0e0d9d81d8bb09a4951bae43dd66130e669927a18754ae6a92bee4e69f930d1202f32ead87d07b
0x10001


## Send key


In [792]:
print(hex(A.public_n))
print(hex(A.public_e))
server_encrypted_key = int("0217CC65496F8DABBC92DC8998D8B0B085E91BDCBB9B922E33F3503C43BF3515736458E3E9AF5AEF5D35EA479007C0511347A8936AAFE73EED6F43AE09DA4BF589D3", 16)
server_encrypted_signature = int("06201BA559CF89A44A019C9F9A8DFA8C1B8EF3052368DE1EAC66EEBCC4F1F54198D02D8C81F0B9E5C814CA2E2B353171BF09E2D1ADDB95AB4F765B2E089C395FD5D8",16)
A.Receive_Key(server_encrypted_key,server_encrypted_signature, A.public_e, server_key)

0x8b41386cc56ada854fea1921cec47551132e11c13961a166bf50bd0e0d9d81d8bb09a4951bae43dd66130e669927a18754ae6a92bee4e69f930d1202f32ead87d07b
0x10001
k is not authentic, try again


## Recieve Key

In [794]:
server_key = int("A11EF05C3B5AA7B0F8DF0047D6817BD46F10D47CA89EB22D6312672E04A32E650A7F15313F5ACA38ECA784DE2E38FE20B873EF9063065A93292D7DBD33B4BBFAA9D6AD7B97F1DDE3B5D9CB8E6606872660CF6A64EC73AED9AFD2FE13EDAACE13F3BC5365467F550C017F696E2F1A213F855B139A9F1C93AD92542A654E4D1297", 16)
server_e = 2**16+1
k_1, s_1 = A.Send_Key(server_e, server_key)
print(hex(k_1))
print(hex(s_1))
print(hex(A.public_n))
print(hex(A.public_e))

initial k,  0xa3470535770d1db08ad24ff3881987a6417602cb19288dafff58c8ae041c2a7691295f8a2380a7d697d6fe8cf7a12a0611e23c4f53680c8dcdc5e54b16be5345718
0x57f928399dfc58ad45293c1d2b9b3747340a95534ca0c448c041fee95537010cf937549d7c940f15482320ab838b726bdeb0c3b8e451545a93e03ea4325e421ff864253f1a3520be2e82cc052804889b5375cc246b4c664da94cc8b721e700deafd0f61f1fc492f8861fd3754ff0ad2b00ddbc2104c6c0dffff3953f70b225f4
0x99fe0e3f93858dbf7be46429dd8b6556114b57d2ea7fbf7a7d51d2ed208d477eb199fc2d266ead9bb3a7ff96b6b0adf91e721155e2f00d08f7269a917658c6091d4fac2bf26eae2c57a401508130dcc5e2670ab45c4220e685f5c401f7e6fe7458f7d7093537fef65911160a820dda3b4c96ac49ab850aa7d8de30f3d96bc93c
0x1358de1db5ecbe0813acecb0c9ba63c2a3c2025e6b37b55c4133541576d9af099fb49e0c34faa1babd2fbc6f7ec5ab39c1b734d2a4671354707f008e4f544d5dbfb5
0x10001
